In [1]:
import numpy as np
import pandas as pd
import yaml
import sys
import os
import pickle
from glob import glob
from pathlib import Path
import pickle
import warnings
warnings.filterwarnings('ignore')

sys.path.append('/kaggle/src')
from utils.xgb import fit_xgb
from utils.metric import compute_comptetition_metric
from utils.postprocess import post_process
from utils.set_seed import seed_base
from feature_engineering.stage1 import generate_1st_stage_features

PACKAGE_DIR = Path("/kaggle/src")
CFG = yaml.safe_load(open(PACKAGE_DIR / "config.yaml", "r"))
print(CFG["1st_stage"]["execution"]["exp_id"])

CFG["output_dir"] = f"/kaggle/output/{CFG['1st_stage']['execution']['exp_id']}"
!rm -r {CFG["output_dir"]}
os.makedirs(CFG["output_dir"], exist_ok=True)

seed_base(CFG["env"]["seed"])

2023-10-12 04:59:54.006130: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-12 04:59:54.075856: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-12 04:59:54.533894: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/u

debug_nb1


In [2]:
# 特徴生成
files = glob(f"{CFG['dataset']['step_csv_dir']}/*.csv")
train, features = generate_1st_stage_features(files)

# cv splitとマージ
cv_split = pd.read_csv(CFG['dataset']['cv_split_path'])
train["fold"] = train["series_id"].map(cv_split.set_index("series_id")["fold"])
display(train.head(5))
print(train.shape)

generate features: 100%|██████████| 277/277 [04:57<00:00,  1.07s/it]


,anglez,anglez_amplit_360,anglez_amplit_360_min,anglez_amplit_60,anglez_amplit_60_min,anglez_amplit_720,anglez_amplit_720_min,anglez_diff_360_max,anglez_diff_360_mean,anglez_diff_60_max,...,enmo_min_720,enmo_var_360,enmo_var_60,enmo_var_720,hour,lids,target,step,series_id,fold
0,88.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,1.790645,11.0,99.986168,1,11.5,af91d9a50547,3
1,88.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.783105,0.000000,1.813859,11.0,99.557610,1,35.5,af91d9a50547,3
2,88.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.684932,0.628205,1.483303,11.0,99.026649,1,59.5,af91d9a50547,3
3,88.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.684932,7.538462,1.351724,11.0,98.592644,1,83.5,af91d9a50547,3
4,88.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.013699,0.000000,1.351724,11.0,98.226341,1,107.5,af91d9a50547,3


(5331163, 93)


In [3]:
# 学習
trn_oof, models = fit_xgb(
    X=train, 
    y=train["target"], 
    folds=train["fold"].astype(int), 
    features=features.all_features(),
    params=CFG["1st_stage"]["xgboost"], 
    es_rounds=100,
)
train["oof"] = trn_oof
train = train.drop(columns=features.all_features())
train = train.sort_values(["series_id", "step"]).reset_index(drop=True)

# 保存
for i, model in enumerate(models):
    model.save_model(os.path.join(CFG["output_dir"], f'xgb_fold{i}.model'))
with open(os.path.join(CFG["output_dir"], "features.pkl"), "wb") as f:
    pickle.dump(features, f)
train.to_csv(os.path.join(CFG["output_dir"], "oof.csv"), index=False)

  0%|          | 0/5 [00:00<?, ?it/s]

== fold 0 ==
[0]	eval-logloss:0.61800
[100]	eval-logloss:0.14359
[200]	eval-logloss:0.14106
[300]	eval-logloss:0.14036
[400]	eval-logloss:0.14036
[478]	eval-logloss:0.14073


 20%|██        | 1/5 [00:11<00:47, 11.83s/it]

== fold 1 ==
[0]	eval-logloss:0.61854
[100]	eval-logloss:0.13594
[200]	eval-logloss:0.13179
[300]	eval-logloss:0.13072
[400]	eval-logloss:0.13082
[449]	eval-logloss:0.13091


 40%|████      | 2/5 [00:22<00:33, 11.26s/it]

...

 60%|██████    | 3/5 [00:40<00:28, 14.10s/it]

== fold 3 ==
[0]	eval-logloss:0.61831
[100]	eval-logloss:0.15238
[200]	eval-logloss:0.14930
[300]	eval-logloss:0.14873
[400]	eval-logloss:0.14903
[405]	eval-logloss:0.14902


 80%|████████  | 4/5 [00:50<00:12, 12.51s/it]

== fold 4 ==
[0]	eval-logloss:0.61722
[100]	eval-logloss:0.13821
[200]	eval-logloss:0.13493
[300]	eval-logloss:0.13397
[400]	eval-logloss:0.13340
[500]	eval-logloss:0.13302
[600]	eval-logloss:0.13289
[700]	eval-logloss:0.13276
[800]	eval-logloss:0.13276
[880]	eval-logloss:0.13276


100%|██████████| 5/5 [01:07<00:00, 13.50s/it]


In [4]:
# 後処理
sub = post_process(train)
sub.to_csv(os.path.join(CFG["output_dir"], "submission.csv"), index=False)

# スコア計算
labels = pd.read_csv(f"{CFG['dataset']['competition_dir']}/train_events.csv").dropna()
score, ap_table = compute_comptetition_metric(labels, sub)
print(f"score: {score:.4f}")
display(ap_table)

post process: 100%|██████████| 277/277 [00:47<00:00,  5.86it/s]


score: 0.5134


event   tolerance
onset   12           0.008740
        36           0.088411
        60           0.234192
        90           0.401707
        120          0.550090
        150          0.614920
        180          0.666100
        240          0.717581
        300          0.746268
        360          0.764013
wakeup  12           0.030577
        36           0.211169
        60           0.387835
        90           0.542944
        120          0.617508
        150          0.670464
        180          0.702971
        240          0.747589
        300          0.771596
        360          0.792500
dtype: float64

In [5]:
# 次の候補を決定
next_cand_size = 0
count = 0
next_dict = {}
for series_id, train_df in train.groupby("series_id"):
    train_df = train_df[(train_df["oof"] >= 0.1) & (train_df["oof"] <= 0.9)]
    sub_df = sub[(sub["series_id"] == series_id)]
    label_df = labels[labels["series_id"] == series_id]
    pred_steps = train_df["step"].values
    sub_steps = sub_df["step"].values
    label_steps = label_df["step"].values

    if len(train_df) == 0:
        continue
    next_cand = np.zeros(int(max(max(pred_steps if len(pred_steps) > 0 else [0]), max(sub_steps if len(sub_steps) > 0 else [0]))) + CFG["feature"]["agg_freq"])
    for sub_step in sub_steps:
        next_cand[int(sub_step - CFG["feature"]["agg_freq"] * 10): int(sub_step + CFG["feature"]["agg_freq"] * 10)] = 1
    for pred_step in pred_steps:
        next_cand[int(pred_step - CFG["feature"]["agg_freq"] * 10): int(pred_step + CFG["feature"]["agg_freq"] * 10)] = 1
    next_cand_size += np.sum(next_cand)
    next_dict[series_id] = np.where(next_cand)[0]

    for label_step in label_steps:
        if label_step < next_cand.shape[0]:
            count += next_cand[int(label_step)]
    
recall = count / len(labels)
print(f"recall: {recall:.4f}")
print(f"next_cand_size: {next_cand_size}")

with open(f"{CFG['output_dir']}/next_cands.pkl", "wb") as f:
    pickle.dump(next_dict, f)

recall: 0.9864
next_cand_size: 42295068.0
